# Prompt caching

It is a feature that speeds up Claude's responses and reduces the cost of text generation by reusing computational work from previous requests. Instead of throwing away all the processing work after each request, Claude can save and reuse it when you send similar content again.

In [1]:
from anthropic import Anthropic
from anthropic.types import Message
from dotenv import load_dotenv
import os   
load_dotenv()

import logging
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(name)s - %(levelname)s - %(message)s',
    datefmt='%Y-%m-%d %H:%M:%S'
)
logger = logging.getLogger(__name__)

try:
    client = Anthropic(
        api_key=os.getenv("ANTHROPIC_API_KEY"),
    )
    logger.info("Anthropic client created successfully.")
except Exception as e:
    logger.error(f"Failed to create Anthropic client: {e}")
    raise

2025-08-10 22:13:30 - __main__ - INFO - Anthropic client created successfully.


In [2]:
MODEL="claude-3-7-sonnet-20250219"
TEMPERATURE=1.0

In [ ]:
def add_user_message(messages, message_content):
    if isinstance(message_content, list):
        user_message = {
            "role": "user",
            "content": message_content
        }
    else:
        user_message = {
            "role": "user",
            "content": [{"type": "text", "text": str(message_content)}]
        }

    messages.append(user_message)


def add_assistant_message(messages, message_content):
    if isinstance(message_content, Message):
        assistant_message = {
            "role": "assistant",
            "content": message_content
        }
    else:
        assistant_message = {
            "role": "assistant",
            "content": [{"type": "text", "text": message_content}]
        }

    messages.append(assistant_message)

def chat(messages, model=MODEL, temperature=TEMPERATURE, system=None, stop_sequences=None, thinking=False, thinking_budget=1024, tools=None):
    try:
        params = {
            "model": model,
            "max_tokens": 4000,
            "messages": messages,
            "temperature": temperature,
            "stop_sequences": stop_sequences,
        }
        
        if thinking:
            params["thinking"] = {
                "type": "enabled",
                "budget_tokens": thinking_budget
            }
        
        if tools:
            cached_tools = tools.copy()
            if cached_tools:
                cached_tools[-1] = {
                    **cached_tools[-1],
                    "cache_control": {"type": "ephemeral"}
                }
            params["tools"] = cached_tools

        if system:
            params["system"] = [
                {
                    "type": "text", 
                    "text": system,
                    "cache_control": {"type": "ephemeral"}
                }
            ]

        return client.messages.create(**params)
    
    except Exception as e:
        logger.error(f"Chat failed: {e}")
        raise e
    
def text_from_message(message):
    return "\n".join(
        [block.text for block in message.content if block.type == "text"]
    )

In [4]:
# Detailed API Documentation Prompt (~1400 tokens
API_DOCUMENTATION_PROMPT = """
# REST API Design Guidelines and Best Practices

You are an expert API architect and technical documentation specialist. Your role is to provide comprehensive guidance on designing, implementing, and documenting RESTful APIs that follow industry best practices and modern standards.

## Core Responsibilities

When users ask for API design advice, implementation guidance, or documentation help, you should:

1. **Design Principles**: Apply RESTful design principles including proper HTTP method usage, resource-based URLs, stateless communication, and appropriate status codes.

2. **Security Standards**: Recommend authentication mechanisms (OAuth 2.0, JWT, API keys), authorization patterns, rate limiting, input validation, and data encryption best practices.

3. **Documentation Excellence**: Create clear, comprehensive API documentation including endpoint descriptions, request/response examples, parameter definitions, error handling, and usage guidelines.

4. **Performance Optimization**: Suggest caching strategies, pagination techniques, efficient data serialization, compression methods, and database query optimization.

5. **Error Handling**: Design consistent error response formats, meaningful error messages, proper HTTP status codes, and comprehensive error documentation.

## Response Format Guidelines

Structure your responses with:
- **Overview**: Brief summary of the recommendation or solution
- **Implementation Details**: Step-by-step technical guidance with code examples
- **Best Practices**: Industry standards and conventions to follow
- **Security Considerations**: Potential vulnerabilities and mitigation strategies
- **Testing Approach**: How to validate and test the implementation
- **Documentation Template**: Sample documentation format when applicable

## Technical Standards

Follow these standards in your recommendations:
- HTTP/1.1 and HTTP/2 compatibility
- JSON as primary data format with XML support when needed
- OpenAPI 3.0 specification compliance
- Semantic versioning for API versions
- ISO 8601 for date/time formats
- UTF-8 encoding for all text data
- HTTPS for all production endpoints

## Code Examples

Provide practical examples in multiple programming languages when relevant:
- Node.js/Express for backend implementations
- Python/FastAPI or Flask for API servers
- cURL commands for testing
- JavaScript/Fetch for client-side integration
- Postman collection formats

Focus on creating APIs that are intuitive, maintainable, scalable, and developer-friendly. Always consider both the API provider's operational needs and the consumer's integration experience.
"""

In [5]:
# Tool Schemas, ~1.7k tokens
from anthropic.types import ToolParam

add_duration_to_datetime_schema = ToolParam(
    {
        "name": "add_duration_to_datetime",
        "description": "Add a specified duration to a datetime string and returns the resulting datetime in a detailed format. This tool converts an input datetime string to a Python datetime object, adds the specified duration in the requested unit, and returns a formatted string of the resulting datetime. It handles various time units including seconds, minutes, hours, days, weeks, months, and years, with special handling for month and year calculations to account for varying month lengths and leap years. The output is always returned in a detailed format that includes the day of the week, month name, day, year, and time with AM/PM indicator (e.g., 'Thursday, April 03, 2025 10:30:00 AM').",
        "input_schema": {
            "type": "object",
            "properties": {
                "datetime_str": {
                    "type": "string",
                    "description": "The input datetime string to which the duration will be added. This should be formatted according to the input_format parameter.",
                },
                "duration": {
                    "type": "number",
                    "description": "The amount of time to add to the datetime. Can be positive (for future dates) or negative (for past dates). Defaults to 0.",
                },
                "unit": {
                    "type": "string",
                    "description": "The unit of time for the duration. Must be one of: 'seconds', 'minutes', 'hours', 'days', 'weeks', 'months', or 'years'. Defaults to 'days'.",
                },
                "input_format": {
                    "type": "string",
                    "description": "The format string for parsing the input datetime_str, using Python's strptime format codes. For example, '%Y-%m-%d' for ISO format dates like '2025-04-03'. Defaults to '%Y-%m-%d'.",
                },
            },
            "required": ["datetime_str"],
        },
    }
)

set_reminder_schema = ToolParam(
    {
        "name": "set_reminder",
        "description": "Creates a timed reminder that will notify the user at the specified time with the provided content. This tool schedules a notification to be delivered to the user at the exact timestamp provided. It should be used when a user wants to be reminded about something specific at a future point in time. The reminder system will store the content and timestamp, then trigger a notification through the user's preferred notification channels (mobile alerts, email, etc.) when the specified time arrives. Reminders are persisted even if the application is closed or the device is restarted. Users can rely on this function for important time-sensitive notifications such as meetings, tasks, medication schedules, or any other time-bound activities.",
        "input_schema": {
            "type": "object",
            "properties": {
                "content": {
                    "type": "string",
                    "description": "The message text that will be displayed in the reminder notification. This should contain the specific information the user wants to be reminded about, such as 'Take medication', 'Join video call with team', or 'Pay utility bills'.",
                },
                "timestamp": {
                    "type": "string",
                    "description": "The exact date and time when the reminder should be triggered, formatted as an ISO 8601 timestamp (YYYY-MM-DDTHH:MM:SS) or a Unix timestamp. The system handles all timezone processing internally, ensuring reminders are triggered at the correct time regardless of where the user is located. Users can simply specify the desired time without worrying about timezone configurations.",
                },
            },
            "required": ["content", "timestamp"],
        },
    }
)


get_current_datetime_schema = ToolParam(
    {
        "name": "get_current_datetime",
        "description": "Returns the current date and time formatted according to the specified format string. This tool provides the current system time formatted as a string. Use this tool when you need to know the current date and time, such as for timestamping records, calculating time differences, or displaying the current time to users. The default format returns the date and time in ISO-like format (YYYY-MM-DD HH:MM:SS).",
        "input_schema": {
            "type": "object",
            "properties": {
                "date_format": {
                    "type": "string",
                    "description": "A string specifying the format of the returned datetime. Uses Python's strftime format codes. For example, '%Y-%m-%d' returns just the date in YYYY-MM-DD format, '%H:%M:%S' returns just the time in HH:MM:SS format, '%B %d, %Y' returns a date like 'May 07, 2025'. The default is '%Y-%m-%d %H:%M:%S' which returns a complete timestamp like '2025-05-07 14:32:15'.",
                    "default": "%Y-%m-%d %H:%M:%S",
                }
            },
            "required": [],
        },
    }
)


db_query_schema = ToolParam(
    {
        "name": "db_query",
        "description": "Executes SQL queries against a SQLite database and returns the results. This tool allows running SELECT, INSERT, UPDATE, DELETE, and other SQL statements on a specified SQLite database. For SELECT queries, it returns the query results as structured data. For other query types (INSERT, UPDATE, DELETE), it returns metadata about the operation's effects, such as the number of rows affected. The tool implements safety measures to prevent SQL injection and handles errors gracefully with informative error messages. Complex queries are supported, including joins, aggregations, subqueries, and transactions. Results can be formatted in different ways to suit various use cases, such as tabular format for display or structured format for further processing.",
        "input_schema": {
            "type": "object",
            "properties": {
                "query": {
                    "type": "string",
                    "description": "The SQL query to execute against the database. Can be any valid SQLite SQL statement including SELECT, INSERT, UPDATE, DELETE, CREATE TABLE, etc.",
                },
                "database_path": {
                    "type": "string",
                    "description": "The path to the SQLite database file. If not provided, the default database configured in the system will be used.",
                },
                "params": {
                    "type": "object",
                    "description": "Parameters to bind to the query for parameterized statements. This should be a dictionary where keys correspond to named parameters in the query (e.g., {'user_id': 123} for a query containing ':user_id'). Using parameterized queries is highly recommended to prevent SQL injection.",
                },
                "result_format": {
                    "type": "string",
                    "description": "The format in which to return query results. Options are 'dict' (list of dictionaries, each representing a row), 'list' (list of lists, first row contains column names), or 'table' (formatted as an ASCII table for display). Defaults to 'dict'.",
                    "enum": ["dict", "list", "table"],
                    "default": "dict",
                },
                "max_rows": {
                    "type": "integer",
                    "description": "The maximum number of rows to return for SELECT queries. Use this to limit result size for queries that might return very large datasets. A value of 0 means no limit. Defaults to 1000.",
                    "default": 1000,
                },
                "transaction": {
                    "type": "boolean",
                    "description": "Whether to execute the query within a transaction. If true, the query will be wrapped in BEGIN and COMMIT statements, allowing for rollback in case of errors. Defaults to false for SELECT queries and true for other query types.",
                    "default": False,
                },
            },
            "required": ["query"],
        },
    }
)

tools = [
    add_duration_to_datetime_schema,
    set_reminder_schema,
    get_current_datetime_schema,
    db_query_schema
]

In [ ]:
def main():
    messages = []
    logger.info("Starting chat session with API documentation prompt.")
    add_user_message(messages, "Design a RESTful API for a library management system")
    response = chat(messages, system=API_DOCUMENTATION_PROMPT, tools=tools)
    
    logger.info(f"Response generated successfully.")
    
    return response

In [7]:
results1 = main()

# Second call - should use cache (faster)
results2 = main() 

# Check usage statistics to see cache hits
print(f"Usage 1: {results1.usage}")
print(f"Usage 2: {results2.usage}")

2025-08-10 22:15:03 - httpx - INFO - HTTP Request: POST https://api.anthropic.com/v1/messages "HTTP/1.1 200 OK"
2025-08-10 22:15:03 - __main__ - INFO - Response generated successfully.
2025-08-10 22:15:46 - httpx - INFO - HTTP Request: POST https://api.anthropic.com/v1/messages "HTTP/1.1 200 OK"
2025-08-10 22:15:46 - __main__ - INFO - Response generated successfully.


Usage 1: Usage(cache_creation_input_tokens=2293, cache_read_input_tokens=0, input_tokens=152, output_tokens=3465, server_tool_use=None, service_tier='standard')
Usage 2: Usage(cache_creation_input_tokens=0, cache_read_input_tokens=2293, input_tokens=152, output_tokens=3012, server_tool_use=None, service_tier='standard')


In [9]:
print(text_from_message(results1))

# RESTful API Design for Library Management System

## Overview
I'll design a comprehensive RESTful API for a library management system that allows managing books, patrons, loans, reservations, and other library resources. This API will follow REST principles with proper resource modeling, HTTP methods, status codes, and security considerations.

## Resource Modeling

### Core Resources

1. **Books** - Physical and digital books in the library collection
2. **Authors** - Information about book authors
3. **Patrons** - Library users/members
4. **Loans** - Records of books borrowed by patrons
5. **Reservations** - Requests to borrow books that are currently unavailable
6. **Categories** - Book classifications (fiction, non-fiction, etc.)
7. **Publishers** - Information about book publishers

## API Endpoints

### Books Resource

```
GET /api/v1/books                 # List all books (with pagination)
GET /api/v1/books/{bookId}        # Get a specific book
POST /api/v1/books              